In [2]:
import MDAnalysis as mda
import numpy as np

/home/ricky/anaconda3/envs/all3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ricky/anaconda3/envs/all3/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:59: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [3]:
def get_rotation_of_SDS_molecule(universe,save_path):
    for i in range(150):
        SDS_residue=universe.select_atoms(f"resid {i}")
        if 'SDS' in SDS_residue.resnames:
            S=SDS_residue.select_atoms("name S")
            output_file=f'{save_path}/trajectory_S_COM_SDS_{i}.gro'
            with open(output_file,'w') as f:
                for ts in universe.trajectory:
                    com=SDS_residue.center_of_mass()/10
                    s_coor=S.positions[0]/10
                    f.write(f'Title t= {ts.time:.5f} step= {int(ts.time/0.002)}\n')
                    f.write('    2\n')
                    f.write(f'    1SDS    COM    1{com[0]:8.3f}{com[1]:8.3f}{com[2]:8.3f}\n')
                    f.write(f'    1SDS      S    2{s_coor[0]:8.3f}{s_coor[1]:8.3f}{s_coor[2]:8.3f}\n')
                    f.write('   8.08636   8.08636   8.08636\n')

In [4]:
peptides={
"eYqjD_micelle_50SDS_CHARMM_310K_Na_Neut_OPC_replica",
"GWALP_monomer_micelle_80SDS_CHARMM_310K_Na_Neut_OPC_replica",
"yFis1_micelle_50SDS_CHARMM_310K_Na_Neut_OPC_replica",
"hMff_micelle_50SDS_CHARMM_310K_Na_Neut_OPC_replica",
"eElab_micelle_40SDS_CHARMM_310K_Na_Neut_OPC_replica"
}
path='/media/nenciric/Ricky2020/Corys_peptides/full_trajectories/'

In [ ]:
#generate the 'trajectories' with S-atom -- center of mass vectors, 
#so that correlation functions can be calculated with gromcs 

for peptide in peptides:
    for i in range(4,7):
        pep_path=f'{path}{peptide}{i}/'
        print(pep_path)
        for file in os.listdir(pep_path):
            if file=='README.yaml':
                with open(f'{pep_path}{file}','r') as f:
                    readme=yaml.safe_load(f)
                xtc=f"{pep_path}{readme['FILES_FOR_RELAXATION']['xtc']['NAME']}"
                gro=f"{pep_path}{readme['FILES_FOR_RELAXATION']['gro']['NAME']}"

        universe=mda.Universe(gro,xtc)
        new_dir=f'{peptide}{i}/'
        os.system(f'mkdir {new_dir}')
        get_rotation_of_SDS_molecule(universe,new_dir)
        